## Загрузка данных в Qdrant

In [ ]:
!pip3 install qdrant-client
!pip install -U langchain-qdrant
!pip install 'qdrant-client[fastembed]' --quiet


In [ ]:
import openai
import os
from dotenv import load_dotenv
from openai import OpenAI

from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
from openai import OpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

load_dotenv()   

QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COLLECTION_NAME = "TailSense"

# Initialization
client = OpenAI(api_key=OPENAI_API_KEY)
qdrant = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)


In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
from openai import OpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from qdrant_client.models import PointStruct, VectorParams, Filter, FieldCondition, MatchValue

import hashlib
import os
from datetime import datetime
import uuid

def upload_pdf_to_qdrant(pdf_path: str):
    """
    Загружает PDF-документ в векторную базу данных Qdrant с созданием эмбеддингов через OpenAI.
    
    Функция выполняет следующие операции:
    1. Вычисляет MD5-хэш файла для предотвращения дублирования
    2. Загружает и парсит PDF-документ с помощью PyPDFLoader
    3. Разбивает текст на чанки с перекрытием для лучшего контекста
    4. Создает коллекцию в Qdrant, если она не существует
    5. Проверяет, не был ли файл уже загружен ранее
    6. Генерирует эмбеддинги для каждого чанка через OpenAI API
    7. Сохраняет векторы с метаданными в Qdrant
    
    Args:
        pdf_path (str): Путь к PDF-файлу для загрузки
        
    Returns:
        None
        
    Raises:
        FileNotFoundError: Если указанный PDF-файл не найден
        Exception: При ошибках работы с OpenAI API или Qdrant
        
    Example:
        >>> upload_pdf_to_qdrant("documents/veterinary_guide.pdf")
        📄 Загружено 45 чанков из файла veterinary_guide.pdf
        ✅ Файл успешно загружен в Qdrant
        
    Note:
        - Размер чанка: 1000 символов с перекрытием 200 символов
        - Используется модель text-embedding-3-small для создания эмбеддингов
        - Векторы имеют размерность 1536 и используют косинусное расстояние
    """
    
    filename = os.path.basename(pdf_path)

    # --- Хэш файла для проверки дубликатов ---
    with open(pdf_path, 'rb') as f:
        file_hash = hashlib.md5(f.read()).hexdigest()

    # --- Загружаем PDF ---
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()

    # --- Разбиваем на чанки ---
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_documents(pages)
    print(f"📄 Загружено {len(chunks)} чанков из файла {filename}")

    # --- Создаём коллекцию, если не существует ---
    existing_collections = [c.name for c in qdrant.get_collections().collections]
    if COLLECTION_NAME not in existing_collections:
        print("Создаём новую коллекцию...")
        qdrant.create_collection(
            collection_name=COLLECTION_NAME,
            vectors_config=VectorParams(size=1536, distance="Cosine")
        )
        qdrant.create_payload_index(
            collection_name=COLLECTION_NAME,
            field_name="file_hash",
            field_schema="keyword"
        )
    else:
        # проверяем наличие индекса
        indexes = qdrant.get_collection(COLLECTION_NAME).payload_schema
        if "file_hash" not in indexes:
            qdrant.create_payload_index(
                collection_name=COLLECTION_NAME,
                field_name="file_hash",
                field_schema="keyword"
            )

    # --- Проверяем, был ли файл уже загружен ---
    existing = qdrant.scroll(
        collection_name=COLLECTION_NAME,
        scroll_filter=Filter(
            must=[FieldCondition(key="file_hash", match=MatchValue(value=file_hash))]
        ),
        limit=1
    )[0]

    if existing:
        print(f"⚠️ Файл {filename} уже был загружен. Пропускаем...")
        return

    # --- Генерация эмбеддингов и загрузка в Qdrant ---
    batch_size = 30
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i+batch_size]
        texts = [chunk.page_content for chunk in batch]

        response = client.embeddings.create(
            model="text-embedding-3-small",
            input=texts
        )
        embeddings = [item.embedding for item in response.data]

        # --- Создаём точки для загрузки в Qdrant ---
        points = [
            PointStruct(
                id=str(uuid.uuid4()),
                vector=embeddings[j],
                payload={
                    "text": texts[j],
                    "filename": filename,
                    "file_hash": file_hash,
                    "page_number": batch[j].metadata.get("page", -1),
                    "chunk_index": i + j,
                    "upload_timestamp": datetime.now().isoformat()
                }
            )
            for j in range(len(batch))
        ]

        qdrant.upsert(collection_name=COLLECTION_NAME, points=points)
        print(f"✅ Загружено {len(batch)} чанков ({i+1}–{i+len(batch)}) из {filename}")

    print(f"🎉 Файл {filename} успешно добавлен в Qdrant!")

In [30]:
# === Usage Example ===
upload_pdf_to_qdrant("/Users/timurgattarov/TailSense/data_sourse/papers/_2021.pdf")

📄 Загружено 3089 чанков из файла _2021.pdf
✅ Загружено 30 чанков (1–30) из _2021.pdf
✅ Загружено 30 чанков (31–60) из _2021.pdf
✅ Загружено 30 чанков (61–90) из _2021.pdf
✅ Загружено 30 чанков (91–120) из _2021.pdf
✅ Загружено 30 чанков (121–150) из _2021.pdf
✅ Загружено 30 чанков (151–180) из _2021.pdf
✅ Загружено 30 чанков (181–210) из _2021.pdf
✅ Загружено 30 чанков (211–240) из _2021.pdf
✅ Загружено 30 чанков (241–270) из _2021.pdf
✅ Загружено 30 чанков (271–300) из _2021.pdf
✅ Загружено 30 чанков (301–330) из _2021.pdf
✅ Загружено 30 чанков (331–360) из _2021.pdf
✅ Загружено 30 чанков (361–390) из _2021.pdf
✅ Загружено 30 чанков (391–420) из _2021.pdf
✅ Загружено 30 чанков (421–450) из _2021.pdf
✅ Загружено 30 чанков (451–480) из _2021.pdf
✅ Загружено 30 чанков (481–510) из _2021.pdf
✅ Загружено 30 чанков (511–540) из _2021.pdf
✅ Загружено 30 чанков (541–570) из _2021.pdf
✅ Загружено 30 чанков (571–600) из _2021.pdf
✅ Загружено 30 чанков (601–630) из _2021.pdf
✅ Загружено 30 чанко

Посмотрим пример получения ответа

In [36]:
from openai import OpenAI
from qdrant_client import QdrantClient


def ask_qdrant(question, show_sources=True):
    """Задает вопрос к базе знаний и возвращает ответ с источниками."""
    
    # Создаем эмбеддинг запроса
    query_embedding = client.embeddings.create(
        model="text-embedding-3-small",
        input=question
    ).data[0].embedding

    # Используем новый API query_points
    results = qdrant.query_points(
        collection_name=COLLECTION_NAME,
        query=query_embedding,
        limit=30
    )

    # Собираем контекст и источники
    context_parts = []
    sources = set()
    
    for point in results.points:
        context_parts.append(point.payload["text"])
        if show_sources:
            filename = point.payload.get("filename", "Unknown")
            page = point.payload.get("page_number", -1)
            sources.add(f"{filename} (стр. {page})" if page != -1 else filename)

    context = "\n\n".join(context_parts)

    prompt = f"""Я - ветеринарный эксперт, специализирующийся на заболеваниях животных. Я буду отвечать на вопросы, основываясь ТОЛЬКО на предоставленной информации из контекста, без использования дополнительных знаний.

    Важно: Если в контексте нет достаточной информации для полного ответа на вопрос, я честно укажу на это.

    Контекст:
    {context}

    Вопрос о заболевании животных: {question}

    Ответ (основан только на информации из контекста):"""

    answer = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )

    print("💬 Ответ:\n", answer.choices[0].message.content)
    
    if show_sources and sources:
        print("\n📚 Источники:")
        for source in sorted(sources):
            print(f"  • {source}")


query = "Что такое лепстостироз"
ask_qdrant(query)

💬 Ответ:
 Лептоспироз — это заболевание, вызываемое лептоспирами, грамотрицательными микроорганизмами, которые являются возбудителями инфекции. Лептоспироз характеризуется проникновением возбудителя в организм хозяина и его распространением в органах и тканях. Болезнь может проявляться в различных формах, включая острое или подострое течение, а также имеет выраженную сезонность, с наибольшим числом заболевших в июне, июле и августе. Лептоспиры выделяются больными животными или лептоспироносителями во внешнюю среду с мочой, загрязняя почву и воду, что способствует передаче инфекции другим животным и человеку. Патологические изменения при лептоспирозе часто обнаруживаются в почках, где могут развиваться хронические состояния. Диагноз устанавливается комплексно, включая эпизоотологические данные, клинические проявления и лабораторные исследования.

📚 Источники:
  • SHerbakov_G.G._Kovalev_S._P._YAshin_A.V._Vinnib-ok.org_.pdf (стр. 241)
  • SHerbakov_G.G._Kovalev_S._P._YAshin_A.V._Vinnib-ok

In [ ]:
upload_pdf_to_qdrant("/Users/timurgattarov/TailSense/data_sourse/papers/SHerbakov_G.G._Kovalev_S._P._YAshin_A.V._Vinnib-ok.org_.pdf")

📄 Загружено 2030 чанков из файла SHerbakov_G.G._Kovalev_S._P._YAshin_A.V._Vinnib-ok.org_.pdf
✅ Загружено 30 чанков (1–30) из SHerbakov_G.G._Kovalev_S._P._YAshin_A.V._Vinnib-ok.org_.pdf
✅ Загружено 30 чанков (31–60) из SHerbakov_G.G._Kovalev_S._P._YAshin_A.V._Vinnib-ok.org_.pdf
✅ Загружено 30 чанков (61–90) из SHerbakov_G.G._Kovalev_S._P._YAshin_A.V._Vinnib-ok.org_.pdf
✅ Загружено 30 чанков (91–120) из SHerbakov_G.G._Kovalev_S._P._YAshin_A.V._Vinnib-ok.org_.pdf
✅ Загружено 30 чанков (121–150) из SHerbakov_G.G._Kovalev_S._P._YAshin_A.V._Vinnib-ok.org_.pdf
✅ Загружено 30 чанков (151–180) из SHerbakov_G.G._Kovalev_S._P._YAshin_A.V._Vinnib-ok.org_.pdf
✅ Загружено 30 чанков (181–210) из SHerbakov_G.G._Kovalev_S._P._YAshin_A.V._Vinnib-ok.org_.pdf
✅ Загружено 30 чанков (211–240) из SHerbakov_G.G._Kovalev_S._P._YAshin_A.V._Vinnib-ok.org_.pdf
✅ Загружено 30 чанков (241–270) из SHerbakov_G.G._Kovalev_S._P._YAshin_A.V._Vinnib-ok.org_.pdf
✅ Загружено 30 чанков (271–300) из SHerbakov_G.G._Kovalev_S.

In [33]:
upload_pdf_to_qdrant("/Users/timurgattarov/TailSense/data_sourse/papers/spravochnikveterinarnogospecialistaaahannikov.pdf")
upload_pdf_to_qdrant("/Users/timurgattarov/TailSense/data_sourse/papers/oie_terrestrial_code_vol-1-2021_ru.pdf")

📄 Загружено 1431 чанков из файла spravochnikveterinarnogospecialistaaahannikov.pdf
✅ Загружено 30 чанков (1–30) из spravochnikveterinarnogospecialistaaahannikov.pdf
✅ Загружено 30 чанков (31–60) из spravochnikveterinarnogospecialistaaahannikov.pdf
✅ Загружено 30 чанков (61–90) из spravochnikveterinarnogospecialistaaahannikov.pdf
✅ Загружено 30 чанков (91–120) из spravochnikveterinarnogospecialistaaahannikov.pdf
✅ Загружено 30 чанков (121–150) из spravochnikveterinarnogospecialistaaahannikov.pdf
✅ Загружено 30 чанков (151–180) из spravochnikveterinarnogospecialistaaahannikov.pdf
✅ Загружено 30 чанков (181–210) из spravochnikveterinarnogospecialistaaahannikov.pdf
✅ Загружено 30 чанков (211–240) из spravochnikveterinarnogospecialistaaahannikov.pdf
✅ Загружено 30 чанков (241–270) из spravochnikveterinarnogospecialistaaahannikov.pdf
✅ Загружено 30 чанков (271–300) из spravochnikveterinarnogospecialistaaahannikov.pdf
✅ Загружено 30 чанков (301–330) из spravochnikveterinarnogospecialistaaahan